##### Compute SOFA score

Connect to database

In [9]:
import __init__

In [11]:
import numpy as np
import pandas as pd
import datetime

# Open a connection to OMOP
from fleming_lib.tools import connect_to_omop
from fleming_lib.severity_scores import compute_sofa_score
conn = connect_to_omop()

In [12]:
patient_id = (62063368, 62106569)  # person_id

In [17]:
# Get data

query = """
select
    distinct m.person_id, m.measurement_datetime,m.measurement_concept_id, m.measurement_concept_name, m.value_source_value, m.unit_source_value
from 
    measurement m 

where
    m.measurement_concept_id in
    (3024929,   -- platelets
     3016502,   -- pa02
     3024128,   -- bilirubin
     3027598,   -- map_bp
     3016723 ,  -- creatinine
     3032652    -- glasgow coma scale
    )
  
    
and m.person_id in {}
order by m.person_id, m.measurement_datetime
;""".format(patient_id)

measures = pd.read_sql_query(query, conn)
measures.iloc[:10]

,person_id,measurement_datetime,measurement_concept_id,measurement_concept_name,value_source_value,unit_source_value
0,62063368,2188-11-12 07:15:00,3016723,Creatinine serum/plasma,0.5,mg/dL
1,62063368,2188-11-12 07:15:00,3024929,Platelets [#/volume] in Blood by Automated count,720,K/uL
2,62063368,2188-11-12 08:04:00,3024128,Total Bilirubin serum/plasma,0.3,mg/dL
3,62063368,2188-11-12 10:00:00,3027598,Mean blood pressure,107.33300018310547,mmHg
4,62063368,2188-11-12 10:00:00,3032652,Glasgow coma scale,15,points
5,62063368,2188-11-12 10:00:00,3016502,Oxygen saturation in Arterial blood,97,%
6,62063368,2188-11-12 10:30:00,3016502,Oxygen saturation in Arterial blood,98,%
7,62063368,2188-11-12 10:30:00,3027598,Mean blood pressure,110,mmHg
8,62063368,2188-11-12 10:45:00,3016502,Oxygen saturation in Arterial blood,97,%
9,62063368,2188-11-12 10:45:00,3027598,Mean blood pressure,112.66699981689453,mmHg


#### Change type datetime

In [24]:
# Select date range
df=measures
df = df.set_index(['measurement_datetime'])
df=df.loc['2188-11-12 13:00:00':'2188-11-20 13:00:00']

In [27]:
#get features

# Oxygen saturation in Arterial blood
oxygenSat=df[df['measurement_concept_name']=='Oxygen saturation in Arterial blood']
pa02=oxygenSat['value_source_value']
pa02=pa02[0]
pa02=pd.to_numeric(pa02)

# Platelets
platelets=df[df['measurement_concept_name']=='Platelets [#/volume] in Blood by Automated count']
pla=platelets['value_source_value']
pla=pla[0]
pla=pd.to_numeric(pla)

# Total Bilirubin serum/plasma
bilirubin=df[df['measurement_concept_name']=='Total Bilirubin serum/plasma']
bil=bilirubin['value_source_value']
bil=bil[0]
bil=pd.to_numeric(bil)

# Mean blood pressure
mean_blood_pressure=df[df['measurement_concept_name']=='Mean blood pressure']
mbp=mean_blood_pressure['value_source_value']
mbp=mbp[0]
mbp=pd.to_numeric(mbp)

# Dopamine
dop=0

# Glasgow coma scale
glasgow_coma_scale=df[df['measurement_concept_name']=='Glasgow coma scale']
gcs=glasgow_coma_scale['value_source_value']
gcs=gcs[0]
gcs=pd.to_numeric(gcs)

# Creatinine
creatinine=df[df['measurement_concept_name']=='Creatinine serum/plasma']
cre=creatinine['value_source_value']
cre=cre[0]
cre=pd.to_numeric(cre)


In [29]:
# Compute SOFA score
score=compute_sofa_score(pa02,pla,bil,mbp,dop,gcs,cre)
print(score)

10


In [22]:
pa02.head()

Series([], Name: value_source_value, dtype: object)

In [30]:
pa02

93